In [ ]:
import shared_variables_robot
import socket
import threading
import CAN_ID
import can
import numpy as np
import time
import json
import math

TRUCS A FAIRE SUR LA RPI POUR GERER LE CAN

aller dans le fichier config : sudo nano /boot/config.txt ou sudo nano /boot/firmware/config.txt si pas trouvé dans juste boot
dans config.txt aller desencdre jusqu'à [all] 
une fois dans [all] écrire les choses suivantes :
- dtparam=spi=on
- dtoverlay=mcp2515-can0,oscillator=12000000,interrupt=25,spimaxfrequency=2000000
Normalement il n'y a rien d'autre à mettre pour la rpi 5. Cela dit pour la rpi 4 ajouter : dtoverlay=spi-bcm2712

Pour activer can0:
sudo ip link set can0 up type can bitrate 1000000
Ensuite pour vérifier :
ip link show -> regarder si le can0 est up

In [ ]:
main_loop_event = threading.Event()
depot_event = threading.Event()
swomp_event = threading.Event()
waypoints_lock = threading.Lock()
launched_lock = threading.Lock()

def start_wifi():

    PORT = 5050
    ADDR = (shared_variables_robot.ip_dico["Matthieu"],PORT)
    client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client.connect(ADDR)
    client.settimeout(0.5)

    shared_variables_robot.client = client

def receive():

    received_data = None
    prec_waypoints = None
    
    while main_loop_event.is_set():
        with shared_variables_robot.client_lock:
            try :
                received_data = shared_variables_robot.client.recv(1024).decode('utf-8')
            except socket.timeout:
                received_data = None
                continue
            
            if received_data:
                if received_data[0] == "[":
                    with waypoints_lock:
                        shared_variables_robot.waypoints = json.loads(received_data)
                        local_waypoints = shared_variables_robot.waypoints

                    if local_waypoints != prec_waypoints:
                        compteur = 0
                        for points in local_waypoints:
                            
                            send_CAN([points,compteur,len(local_waypoints)])
                            compteur += 1
                            
                        prec_waypoints = local_waypoints
                        received_data = None
                    
                    with shared_variables_robot.front_state_lock:
                        local_full_front = shared_variables_robot.front["full"]

                    with shared_variables_robot.back_state_lock:
                        local_full_back = shared_variables_robot.back["full"]

                    if not local_full_front and not local_full_back:
                        time.sleep(0.3)
                        send_CAN(CAN_ID.CONSTRUIRE_AVANT_PREPARER)
                        send_CAN(CAN_ID.CONSTRUIRE_ARRIERE_PREPARER)

                else:
                    if received_data == "STOP_RPI":
                        main_loop_event.clear()
                        client.close()
                    if received_data == "STOP":
                        send_CAN(CAN_ID.STOP_ROBOT_FIN_MATCH)
                    if received_data == "START":
                        main_loop_event.set()
            

def send():

    can_msg = None
    theta = None

    while main_loop_event.is_set():
        can_msg, theta = receive_CAN(can_msg,theta)

        with waypoints_lock:
                if not shared_variables_robot.waypoints:
                    last_point = None
                else:
                    last_point = shared_variables_robot.waypoints[-1]

        if last_point in shared_variables_robot.depot_points and theta is not None and not depot_event.is_set(): #point
            shared_variables_robot.robot_angle = theta
            depot_event.set()
            t_build = threading.Thread(target=gestion_build, args=(last_point,))
            t_build.start()

        elif last_point in shared_variables_robot.swomp_points and theta is not None and not swomp_event.is_set(): #point de récup
            shared_variables_robot.robot_angle = theta
            swomp_event.set()
            t_swomp = threading.Thread(target=gestion_swomp, args=(last_point,theta))
            t_swomp.start()


def gestion_build(last_point):
    global client

    finished = False
    can_msg = None
    _ = None
    local_front_state = False
    local_back_state = False
    
    while local_front_state != True and local_back_state != True:

        with shared_variables_robot.front_state_lock:
            local_front_state = shared_variables_robot.front["build_state"]
        with shared_variables_robot.back_state_lock:
            local_back_state = shared_variables_robot.back["build_state"]

        time.sleep(0.01)

    front, back = verify_front_back(last_point,shared_variables_robot.robot_angle)
    
    if front in shared_variables_robot.depot_points_front_back and local_front_state == True:
        
        send_CAN(CAN_ID.LACHER_AVANT)
        with shared_variables_robot.front_state_lock:
            shared_variables_robot.front["build_state"] = False
            shared_variables_robot.front["full"] = False

    elif back in shared_variables_robot.depot_points_front_back and local_back_state == True:
        
        send_CAN(CAN_ID.LACHER_ARRIERE)
        with shared_variables_robot.back_state_lock:
            shared_variables_robot.back["build_state"] = False
            shared_variables_robot.back["full"] = False

    elif ((back in shared_variables_robot.depot_points_front_back and local_front_state == True) or
      (front in shared_variables_robot.depot_points_front_back and local_back_state == True)):
        
        while not finished:
            can_msg, _ = receive_CAN(can_msg,_)
            if can_msg is not None:
                finished = True
        
        send_CAN(CAN_ID.LACHER_AVANT if front in shared_variables_robot.depot_points_front_back else CAN_ID.LACHER_ARRIERE)
        
        if front in shared_variables_robot.depot_points_front_back:
            with shared_variables_robot.front_state_lock:
                shared_variables_robot.front["build_state"] = False
                shared_variables_robot.front["full"] = False
        else:
            with shared_variables_robot.back_state_lock:
                shared_variables_robot.back["build_state"] = False
                shared_variables_robot.back["full"] = False

    with shared_variables_robot.client_lock:
        message = "Done"
        client.send(message.encode('utf-8'))

    depot_event.clear()

def gestion_swomp(last_point,theta):

    front, back = verify_front_back(last_point,theta)

    if front in shared_variables_robot.swomp_points_front_back:
        send_CAN(CAN_ID.CONSTRUIRE_AVANT_2ETAGE)
        with shared_variables_robot.front_state_lock:
            shared_variables_robot.front["full"] = True

    elif back in shared_variables_robot.swomp_points_front_back:
        send_CAN(CAN_ID.CONSTRUIRE_ARRIERE_2ETAGE)
        with shared_variables_robot.back_state_lock:
            shared_variables_robot.back["full"] = True

    swomp_event.clear()


def send_CAN(msg):
    with shared_variables_robot.CAN_lock:
        if type(msg) == list:

            byte_convert_x = msg[0][0].to_bytes(2, byteorder="big")
            last_x, first_x = byte_convert_x

            byte_convert_y = msg[0][1].to_bytes(2, byteorder="big")
            last_y, first_y = byte_convert_y

            bus.send(can.Message(arbitration_id=CAN_ID.POLAIRE, data=[msg[1],
                                                                       np.uint8(last_x), np.uint8(first_x), np.uint8(last_y),
                                                                       np.uint8(first_y),msg[2],0,0], is_extended_id=False))
            # print("Message envoyé")
            time.sleep(0.015)
        if msg == CAN_ID.STOP_ROBOT_FIN_MATCH:
            bus.send(can.Message(arbitration_id=CAN_ID.STOP_ROBOT_FIN_MATCH, data=[1,0,0,0,0,0,0,0], is_extended_id=False))
        if msg == CAN_ID.CONSTRUIRE_AVANT_2ETAGE:
            bus.send(can.Message(arbitration_id=CAN_ID.CONSTRUIRE_AVANT_2ETAGE, data=[1,0,0,0,0,0,0,0], is_extended_id=False))
        if msg == CAN_ID.CONSTRUIRE_ARRIERE_2ETAGE:
            bus.send(can.Message(arbitration_id=CAN_ID.CONSTRUIRE_ARRIERE_2ETAGE, data=[1,0,0,0,0,0,0,0], is_extended_id=False))
        if msg == CAN_ID.ROTATION:
            deg = 180
            byte_convert_ang = deg.to_bytes(2, byteorder="big")
            fort,faible = byte_convert_ang
            bus.send((can.Message(arbitration_id=CAN_ID.ROTATION, data=[np.uint8(fort),
                                                                        np.uint8(faible),0,0,0,0,0,0], is_extended_id=False)))
            

def receive_CAN(msg,theta):
    with shared_variables_robot.CAN_lock:
        message = bus.recv(timeout=0.015)
        if message is None:
            return None, None
        else:
            if message.arbitration_id == CAN_ID.ACKNOWLEDGE_BASE_ROULANTE:
                return True, theta
            if message.arbitration_id == CAN_ID.ODO_SEND:
                pfort,pfaible = message.data[4],message.data[5]
                return msg, (pfort<<8) | pfaible
            if message.arbitration_id == CAN_ID.CONSTRUIRE_AVANT_TERMINEE:
                with shared_variables_robot.front_state_lock:
                    shared_variables_robot.front["build_state"] = True
            if message.arbitration_id == CAN_ID.CONSTRUIRE_ARRIERE_TERMINEE:
                with shared_variables_robot.back_state_lock:
                    shared_variables_robot.back["build_state"] = True
             
        return msg, theta
    
def verify_front_back(mid,theta):
    return(
        (mid[0]+139*math.cos(math.radians(theta)),mid[1]+139*math.sin(math.radians(theta))),
        (mid[0]-139*math.cos(math.radians(theta)),mid[1]-139*math.sin(math.radians(theta))))


if __name__ == "__main__":
    bus = can.interface.Bus(channel='can0', interface='socketcan', bitrate=1000000)
    #bus.send(can.Message(arbitration_id=CAN_ID.STOP_ROBOT_FIN_MATCH, data=[1,0,0,0,0,0,0,0], is_extended_id=False))                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

    start_wifi()

    main_loop_event.set()

    t_receive = threading.Thread(target = receive)
    t_send = threading.Thread(target = send)

    t_receive.start()
    t_send.start()  

    t_receive.join()
    t_send.join()

Message envoyé
Message envoyé
Message envoyé
Message envoyé
Message envoyé
Message envoyé
Message envoyé
Message envoyé
Message envoyé
Message envoyé
Message envoyé
Message envoyé
Message envoyé
Message envoyé
Message envoyé
Message envoyé
Message envoyé
Message envoyé
Message envoyé
Message envoyé
